<a href="https://colab.research.google.com/github/heejvely/third-project/blob/main/Etri_et5_train_290000_extra_training_ver10_validation_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고 url : https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

In [14]:
# mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# module install 
!pip install transformers
!pip install sentencepiece==0.1.91

런타임 다시 시작 눌러야할 수 있음  
만일 tokenizer Nonetype 에러시 런타임 다시 시작

In [16]:
# model.generate(pieces)
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

# model folder path
model_folder = '/content/drive/MyDrive/colab/3차/pretrained_train_290000_extra_training_ver10/'

model = T5ForConditionalGeneration.from_pretrained(model_folder)
tokenizer = T5Tokenizer.from_pretrained(model_folder)

In [17]:
# gpu로 지정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
class CustomDataset:

# """
# author : Author of the article
# date : Date the article was published
# headline: Headline for the published article
# read_more : URL for the article to follow online
# text: This is the summary of the article
# ctext: This is the complete article
# """

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in tqdm(enumerate(loader, 0)):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            pass
            
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [19]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [20]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(45100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(45100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=768, out_features=3072, bias=False)
              (wi_1): Linear(in_features=768, out_features=3072, bias=False)
              (wo)

hyper-parameters

In [21]:
config = T5Config()
config.MAX_LEN = 1024
config.SUMMARY_LEN = 150 
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 15       # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)

In [22]:
train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

In [23]:
!nvidia-smi

Mon Dec  6 07:48:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    38W / 300W |   5051MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/valid.csv')[['document','label']]
valid = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/valid_10000.csv')[['document','label']]
train_dataset = dataset[:20000]
validation_dataset = valid

partial dataset


In [25]:
import numpy as np

train_dataset = train_dataset
validation_dataset = validation_dataset

train

In [ ]:
train_dataset.columns = ['ctext','text']
train_dataset.ctext = 'summarize: ' + train_dataset.ctext

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

training_loader = DataLoader(training_set, **train_params)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
for epoch in range(config.TRAIN_EPOCHS):
    print (1)
    train(epoch, tokenizer, model, device, training_loader, optimizer)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


1


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  4.3824262619018555


501it [02:42,  3.09it/s]

...

9001it [48:17,  3.09it/s]

Epoch: 14, Loss:  0.032430898398160934


9501it [50:58,  3.09it/s]

Epoch: 14, Loss:  0.12319434434175491


10000it [53:38,  3.11it/s]


In [ ]:
# model save
tokenizer.save_pretrained('/content/drive/MyDrive/colab/3차/pretrained_train_290000_extra_training_ver10/')
model.save_pretrained('/content/drive/MyDrive/colab/3차/pretrained_train_290000_extra_training_ver10/')

valid(test)

In [ ]:
# model.generate(pieces)
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

# model folder path
model_folder = '/content/drive/MyDrive/colab/3차/pretrained_train_290000_extra_training_ver10/'

model = T5ForConditionalGeneration.from_pretrained(model_folder)
tokenizer = T5Tokenizer.from_pretrained(model_folder)

In [26]:
validation_dataset.columns = ['ctext','text']
validation_dataset.ctext = 'summarize: ' + validation_dataset.ctext

val_set = CustomDataset(validation_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

val_loader = DataLoader(val_set, **val_params)

for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

final_df

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 700
Completed 800
Completed 900
Completed 1000
Completed 1100
Completed 1200
Completed 1300
Completed 1400
Completed 1500
Completed 1600
Completed 1700
Completed 1800
Completed 1900
Completed 2000
Completed 2100
Completed 2200
Completed 2300
Completed 2400
Completed 2500
Completed 2600
Completed 2700
Completed 2800
Completed 2900
Completed 3000
Completed 3100
Completed 3200
Completed 3300
Completed 3400
Completed 3500
Completed 3600
Completed 3700
Completed 3800
Completed 3900
Completed 4000
Completed 4100
Completed 4200
Completed 4300
Completed 4400
Completed 4500
Completed 4600
Completed 4700
Completed 4800
Completed 4900


,Generated Text,Actual Text
0,구의 메카 충북 단양군에서 탁구인들이 그동안 갈고 배운 자웅을 겨루는 '제3회 만천...,9~10일 이틀간 충북 단양국민체육센터에서 '제3회 만천하스카이워크배 전국오픈 탁구...
1,시는 미세먼지 저감을 위해 시행 중인 차량운행제한 정책의 실효성을 높이기 위해 내년...,"'승용차마일리지제'는 서울시가 지난 2017년 도입한 것으로, 미세먼지 저감을 위한..."
2,"면에서 상어 1개체가 발견되었으며, 최근 제주도내 해수욕장에 상어가 나타나면서 제주...","최근 제주도내 해수욕장에서 발생한 상어 출현으로, 제주특별자치도는 10일 상어퇴치기..."
3,홍준표 전 대표는 문희상 국회의장이 패스트트랙 법안을 본회의에 부의하려는 것을 두고...,지난달 30일 홍준표 자유한국당 전 대표가 문희상 의장이 패스트트랙 법안을 본회의에...
4,에서의 삼성 SDS의 대표 홍원표 대표는 지난 25일 서울 송파구 본사에서 중국 I...,"삼성SDS(대표 홍원표)는 지난 25일 서울 송파구 본사에서 시스템통합, IT아웃소..."
...,...,...
9995,이노베이션은 지난해 4·4분기부터 '그룹 통합 채용사이트'와 별도로 'SK이노베이션...,SK이노베이션은 김준 SK이노베이션 총괄 사장의 인재 영입 철학을 반영하여 작년 4...
9996,&G는 내달 7일까지 취업을 준비하는 2~4학년 대학생 150명을 대상으로 취업 상...,내달 7일까지 KT&G는 2~4학년 대학생 150명을 대상으로 취업 상담 프로그램 ...
9997,원이 조선업이 위기를 맞으며 지역 상권도 식품하고 있는 '거제도' 첫 편으로 꾸며진...,지난 27일 '백종원의 골목식당'은 '보리밥&코다리찜'의 주방점검 장면에서 동시간대...
9998,하는 자녀들에게 수능 당일 가족들이 꼭 챙겨주는 소지품 중 하나다.,소의 도축 과정에서 발견하여 약재로 쓰는 우황은 기본적으로 몸 안의 열을 치료하는 ...


In [27]:
final_df.to_csv('/content/drive/MyDrive/colab/3차/dataset/summarization_et5_train_290000_extra_training_ver10_1.csv')

In [28]:
df = pd.read_csv('/content/drive/MyDrive/colab/3차/dataset/summarization_et5_train_290000_extra_training_ver10_1.csv', index_col = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Generated Text  10000 non-null  object
 1   Actual Text     10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


In [29]:
df[df['Generated Text'].isna()]

,Generated Text,Actual Text


In [38]:
df.loc[119, 'Generated Text']

'로봇 사업을 미래 성장동력으로 삼고 다양한 분야에서 사업을 추진하고 있는 LG전자와 CJ푸드빌은 18일 서울시 중구에 위치한 CJ푸드빌 본사에서 LG전자 로봇사업센터 노진서 전무, CJ푸드빌 전략기획담당 오광석 인사 등 양사 관계자들이 참석한 가운데 업무협약을 체결했다고 밝혔다.'

In [39]:
df.loc[8217, 'Generated Text']

'싱크홀이 발생한 주변 15m 가량은 아스팔트가 심하게 변형돼 있어, 전주시 시민들이 불안을 호소하고 있다.'

In [30]:
df.to_csv('/content/drive/MyDrive/3차 프로젝트/summarization_et5_train_290000_extra_training_ver10_1.csv', index = False)